In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model
from sklearn import cross_validation
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [8]:
df = pd.read_excel("data0505.xlsx",header=0)
# clean up data
df = df.dropna(how = 'all')
df = df.fillna(0)
df = df.round(4)
df=df[df['Power']>=0]
df.head()

,SOC,SOH,Power,T,SEI_after,SEI_delta
10,80,1.0,0.0526,23,0.0,44.8776
11,80,1.0,0.1579,23,0.0,46.1896
12,80,1.0,0.2632,23,0.0,47.5605
13,80,1.0,0.3684,23,0.0,48.9998
14,80,1.0,0.4737,23,0.0,50.4819


In [10]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df)
df_normalized = pd.DataFrame(np_scaled)
df_normalized.head()

,0,1,2,3,4,5
0,1.0,0.0,0.000000,0.0,0.0,0.760108
1,1.0,0.0,0.111146,0.0,0.0,0.782330
2,1.0,0.0,0.222293,0.0,0.0,0.805549
3,1.0,0.0,0.333333,0.0,0.0,0.829927
4,1.0,0.0,0.444480,0.0,0.0,0.855030


In [11]:
x = np.array(df_normalized.ix[:,0:2])#first three column are SoC, SoH, power
y = np.array(df_normalized.ix[:,5])#delta SEI
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(
x, y, test_size=0.2, random_state=42)
total_len = X_train.shape[0]
total_len

4880

In [12]:
# Parameters
learning_rate = 0.001
training_epochs = 50
batch_size = 100
display_step = 1
dropout_rate = 0.1
# Network Parameters
n_hidden_1 = 10 # 1st layer number of features
n_hidden_2 = 5 # 2nd layer number of features
n_input = X_train.shape[1]
n_classes = 1

# tf Graph input
x = tf.placeholder("float", [None, 3])
y = tf.placeholder("float", [None])

In [13]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [14]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], 0, 0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], 0, 0.1))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], 0, 0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_classes], 0, 0.1))
}

In [15]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

In [16]:
# Define loss and optimizer
cost = tf.reduce_mean((tf.transpose(pred)-y)*(tf.transpose(pred)-y)) 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    tf.initialize_all_variables()

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(total_len/batch_size)
        # Loop over all batches
        for i in range(total_batch-1):
            batch_x = X_train[i*batch_size:(i+1)*batch_size]
            batch_y = Y_train[i*batch_size:(i+1)*batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, p = sess.run([optimizer, cost, pred], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        # sample prediction
        label_value = batch_y
        estimate = p
        err = label_value-estimate
        print ("num batch:", total_batch)

        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print ("[*]----------------------------")
            for i in range(3):
                print ("label value:", label_value[i], \
                    "estimated value:", estimate[i])
            print ("[*]============================")

    print ("Optimization Finished!")
    
    # Test model
    # correct_prediction = tf.equal(tf.argmax(pred,0), tf.argmax(y,0))
    # Calculate accuracy
    accuracy = tf.reduce_mean((tf.transpose(pred)-y)*(tf.transpose(pred)-y)) 
    print ("MSE:", accuracy.eval({x: X_test, y: Y_test}))

num batch: 48
Epoch: 0001 cost= 0.653324438
[*]----------------------------
label value: 0.844460892497 estimated value: [ 0.07771525]
label value: 0.788510037923 estimated value: [ 0.08068381]
label value: 0.75568544624 estimated value: [ 0.07602236]
[*]============================
num batch: 48
Epoch: 0002 cost= 0.424272126
[*]----------------------------
label value: 0.844460892497 estimated value: [ 0.35695586]
label value: 0.788510037923 estimated value: [ 0.36942384]
label value: 0.75568544624 estimated value: [ 0.32976228]
[*]============================
num batch: 48
Epoch: 0003 cost= 0.094070199
[*]----------------------------
label value: 0.844460892497 estimated value: [ 0.82547671]
label value: 0.788510037923 estimated value: [ 0.85426039]
label value: 0.75568544624 estimated value: [ 0.74013388]
[*]============================
num batch: 48
Epoch: 0004 cost= 0.010735776
[*]----------------------------
label value: 0.844460892497 estimated value: [ 0.88168484]
label value: 